<a href="https://colab.research.google.com/github/a19310422/Practicas-19310422/blob/main/Chatbot_19310422.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 **CHATBOT IA 19310422**

Se utilizo un dataset creado por mi 

In [ ]:
from google.colab import files #Aqui se importa la funcion para subir un archivo
uploaded = files.upload() #Aqui se sube el archivo como tal, que es el intents.json

Saving intents.json to intents.json


In [ ]:
#se instala el tensorflow
!pip install tensorflow==1.15
!pip install tflearn

In [ ]:
import numpy as np
import tensorflow as tf

import tflearn
import random
import pickle
import json #se importa la libreria json

with open('intents.json') as file: #se abre el archivo que cree
    intents = json.load(file, strict = False) #y se obtioenen los intents del data set
intents = intents['intents'] # Se obtiene los intents individuales del dataset

#Tendiamos un loop (for anidado)
print("[", end = "")
for intent in intents:
  print("{", end = "")
  for key, value in intent.items():
    print("{}: {},".format(key, value))
  print("\b\b\n},")
print("\b\b]") #Y con esto se muestran los posibles matches 

import nltk
nltk.download('all') #esta libreria se utilizara para entrenar nuestro chatbot

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english') #en el formato de ingles, y esto ayudara a porcesar todos los datos

retrain_model = True

if retrain_model: #aca entra en un if donde carna las listas
    all_words = [] #Aca carga todas las palabras de todos nuestros pattenrs
    all_tags = [] #Cargara todos los tags asocioiados a cada intents
    intent_patterns = [] #Se crea una lista con todos los pattenrs para cada intencion donde cada pattern se vielve individual
    intent_tags = [] #Lista relacionada con intent_patterns donde cada intent partner esta relacionado al que corresponde
    
    #se entra al for
    for intent in intents: #uno de intent-intents
        for pattern in intent['patterns']: #y esta para pattern en cada intent
            words = nltk.word_tokenize(pattern) #se rrellena todas las listas que creamos arriba, con las palabras de cada pattern
                                                #se divide el pattern con palabras individuales

            all_words.extend(words)
            intent_patterns.append(words)
            intent_tags.append(intent['tag']) 
        all_tags.append(intent['tag'])
      
    #Aca las palabras se separan para que no se confundan las palabras mas perecidas entre ellas(claro que en ingles)
    #con esto el chatbot no se confunde
    all_words = [stemmer.stem(word.lower()) for word in all_words]
    all_words = sorted(list(set(all_words)))
    
    all_tags = sorted(all_tags)
    x_train = []
    y_train = []
    y_empty = [0 for i in range(len(all_tags))]
    
    #Se crea nuestros valores de entrada y salida del conjunto de entrenamiento para nuestro algoritmo de aprendizaje profundo
    for index, intent in enumerate(intent_patterns): #se manda como parametro a intent_patterns
        bag_of_words = []#Y se guarda como en una bolsa o vector
        
        intent_words = [stemmer.stem(word.lower()) for word in intent]
        #se crea los valores de entrada y salida del conjunto
        #se va iterando con el iterns y se va convirtiendo a los vectores
        for word in all_words:
            if word in intent_words:
                bag_of_words.append(1)
            else:
                bag_of_words.append(0)
                
        one_hot_encode_y = y_empty[:]
        one_hot_encode_y[all_tags.index(intent_tags[index])] = 1
        
        #Y esta inf se va a guardar en los modelos a entrenat 'x=intents' 'y=los valores que van a estar asociados a la bolsa'. 
        x_train.append(bag_of_words)
        y_train.append(one_hot_encode_y)
    
    #Estos son los datos que usaremos para entrenar nuestra red neuronal más adelante
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    
    #Aquí solo guardamos nuestros datos de entrenamiento para que no necesitemos procesarlos 
    #nuevamente si solo queremos ejecutar nuestro chatbot
    with open('training_data.pickle', 'wb') as f:
        pickle.dump((all_words, all_tags, x_train, y_train), f)
else:
    with open('training_data.pickle', 'rb') as f:
        all_words, all_tags, x_train, y_train = pickle.load(f)

Entonces lo que se hace es que, en lo de bolsa
Cada palabre se convierte en un pattern individual, por lo que cada palabra estara representada en un formato vectorial.
Osea de dos palabras se toman todas las palabras individuales y sera los elementos del vector, y cada frase se representa distinta en el vector, ya que no tendra las mismas palabras individuales. 

In [ ]:
tf.reset_default_graph()

#Crea la red neuronal
neural_net = tflearn.input_data(shape = [None, len(x_train[0])])
neural_net = tflearn.fully_connected(neural_net, 8)
neural_net = tflearn.fully_connected(neural_net, 8)
#Aquí usamos la función de activación softmax para que la salida de nuestra red neuronal sea una probabilidad
neural_net = tflearn.fully_connected(neural_net, len(y_train[0]), activation = 'softmax')
neural_net = tflearn.regression(neural_net)

model = tflearn.DNN(neural_net)

#Aca solo es para tener un respaldo de manera offline, claro que el visual u otra que no sea colab
if retrain_model:
    #Aquí entrenamos la red neuronal con los datos de entrenamiento que creamos en la etapa de PNL
    model.fit(x_train, y_train, n_epoch = 500, batch_size = 8, show_metric = True)
    model.save('model.tfl')
else:
    model.load('./model.tfl')


In [ ]:
#Una vez entrenado, se crea la interfaz para que el usuario interactue con el chatbot
def text_to_bag(text, all_words):
    #Inicialice la bolsa de palabras creando un espacio vacío para cada palabra en el vector
    bag_of_words = [0 for i in range(len(all_words))]
    
    #Primero dividimos la entrada en palabras individuales y 
    #las derivamos para que coincidan con el mismo formato que en nuestro vector
    text_words = nltk.word_tokenize(text)
    text_words = [stemmer.stem(word.lower()) for word in text_words]
    
    #Ahora creamos la bolsa de palabras llenando un 1 para las palabras que usó el usuario
    for word in text_words:
        if word in all_words:
            bag_of_words[all_words.index(word)] = 1
    
    #Y devolver la bolsa de palabras
    return np.array(bag_of_words)



def chat(): #se crea la funcion chat
    #Starting message
    print("Write something to talk to me, to go out just write it down")
    
    #Restablecer el estado del contexto ya que no hay contexto al comienzo de la conversación
    context_state = None
    
    #Esto es lo que respondera por default, por si no entiende lo que le escribimos
    default_responses = ['Sorry, i cant understand you, can you repeat me please?',
                         'I am confuse, lets talk about somthigh elese',
                         'Try whit another word pleas']

    #Este ciclo de chat continuará para siempre hasta que el usuario escriba salir
    while True:
        user_chat = str(input('You: '))
        if user_chat.lower() == 'quit':
            break
        
        user_chat_bag = text_to_bag(user_chat, all_words)

        response = model.predict([user_chat_bag])[0]

        response_index = np.argmax(response)
        response_tag = all_tags[response_index]
        
    
        if response[response_index] > 0.8:
            for intent in intents:
                #Obtener la intención que se predice
                if intent['tag'] == response_tag:
                    #Comprobar si esta respuesta está asociada a un contexto específico
                    if 'context_filter' not in intent or 'context_filter' in intent and intent['context_filter'] == context_state:
                        #Obtenga todas las respuestas posibles de esta intención
                        possible_responses = intent['responses']
                        #Si esta intención está asociada con un conjunto de contexto, establezca el estado del contexto
                        if 'context_set' in intent:
                            context_state = intent['context_set']
                        else:
                            context_state = None
                        #Seleccione un mensaje aleatorio de las respuestas de intención
                        print(random.choice(possible_responses))
                    else:
                        #Se imprime el mensaje preestablecido
                        print(random.choice(default_responses))
        else:
            #Se imprime el mensaje preestablecido
            print(random.choice(default_responses))

In [ ]:
chat()